In [0]:
from pyspark.sql.functions import col, lower, trim, when, regexp_extract, to_date, split, lit, expr, rtrim, ltrim, trim, regexp_replace
from pyspark.sql.types import IntegerType

df = spark.table("soutag")

text_cols = ["title","director","cast","country","listed_in","description"]

df = df
for c in text_cols:
    df = df.withColumn(c, regexp_replace(col(c), r'""+', '"'))

In [0]:
df = df.withColumn("type", trim(lower(df["type"])))\
        .withColumn("title", trim(lower(df["title"])))\
        .withColumn("director", trim(lower(df["director"])))\
        .withColumn("cast", trim(lower(df["cast"])))\
        .withColumn("country", trim(lower(df["country"])))\
        .withColumn("rating", trim(lower(df["rating"])))\
        .withColumn("duration", trim(lower(df["duration"])))\
        .withColumn("listed_in", trim(lower(df["listed_in"])))\
        .withColumn("description", trim(lower(df["description"])))

df.write.mode('overwrite').saveAsTable('soutag')

In [0]:
%sql
CREATE OR REPLACE TABLE silver_soutag AS
SELECT
    TRIM(regexp_replace(show_id, '"', '')) AS show_id,
    TRIM(regexp_replace(type, '"', '')) AS type,
    TRIM(regexp_replace(title, '"', '')) AS title,
    TRIM(regexp_replace(director, '"', '')) AS director,
    TRIM(regexp_replace(cast, '"', '')) AS cast,
    TRIM(regexp_replace(country, '"', '')) AS country,
    TO_DATE(NULLIF(TRIM(regexp_replace(date_added, '"', '')), ''), 'MMMM d, yyyy') AS date_added,
    DATE_FORMAT(
        TO_DATE(NULLIF(TRIM(regexp_replace(date_added, '"', '')), ''), 'MMMM d, yyyy'),
        'dd/MM/yyyy'
    ) AS date_formated,
    CAST(regexp_replace(release_year, '[^0-9]', '') AS INT) AS release_year,
    TRIM(regexp_replace(rating, '"', '')) AS rating,
    TRIM(regexp_replace(duration, '"', '')) AS duration,
    TRIM(regexp_replace(listed_in, '"', '')) AS listed_in,
    TRIM(regexp_replace(description, '"', '')) AS description
FROM soutag;

In [0]:
%sql
CREATE OR REPLACE TABLE titles_clean AS
SELECT
    show_id,
    LOWER(TRIM(type)) AS type,
    LOWER(TRIM(title)) AS title,
    LOWER(TRIM(director)) AS director,
    LOWER(TRIM(cast)) AS cast,
    COALESCE(LOWER(TRIM(country)), 'sem registro') AS country,
    date_added,
    date_formated,
    release_year,
    COALESCE(LOWER(TRIM(rating)), 'not_rated') AS rating,
    CAST(regexp_extract(duration, '^(\\d+)', 1) AS INT) AS duration_value,
    LOWER(regexp_extract(duration, '([A-Za-z]+)$', 1)) AS duration_unit,
    LOWER(TRIM(listed_in)) AS listed_in,
    LOWER(TRIM(description)) AS description
FROM silver_soutag;


In [0]:
%sql
CREATE OR REPLACE TABLE titles_by_country AS
SELECT
    show_id,
    TRIM(country) AS country
FROM titles_clean
LATERAL VIEW explode(split(country, ',')) c AS country_clean;

In [0]:
%sql
CREATE OR REPLACE TABLE titles_by_genre AS
SELECT
    show_id,
    TRIM(listed_in) AS genre
FROM titles_clean
LATERAL VIEW explode(split(listed_in, ',')) g AS genre_clean;